# FIDO - the unified downloader tool for SunPy

### NOTE: Internet access is required in order to use this tutorial

FIDO is a new feature as part of the 0.8 SunPy release. It provides a unified interface to search for and download data from multiple sources and clients. For example, it can be used to search for images via the VSO, or timeseries data via an instrument-specific client, or both simultaneously.

In this tutorial, we will show some examples of how FIDO may be used to both search for and download data of various types.

### Importing modules - getting started

Python is modular, so first we have to import some modules we will need.

In [1]:
import sunpy # import sunpy itself
from sunpy.net import Fido #import the Fido module from sunpy
from sunpy.net import attrs as a #these are the attributes that are used to construct searches with the FIDO client
import matplotlib.pyplot as plt # for plotting
import astropy.units as u # Much of SunPy uses astropy units and quantities

### Prelude - a quick note on AstroPy Quantities

Some Fido search queries require the use of AstroPy quantities. We are not going to cover these in detail here, but a brief explanation is needed.

In short, an Astropy Quantity is just a value attached so some units or dimensions. In the first cell, you will see that we already imported astropy.units and gave it the name u.

#### Simple quantity example

Here is a very quick example showing how quantities can be used. Here we construct a velocity in km/s and a distance in km. When we calculate the time to travel the distance at the given velocity, the result is unit aware. More details of Units and Quantities can be found in other notebooks.

In [2]:
velocity = 100*u.km/u.s
distance = 150000*u.km
time_to_travel = distance/velocity
print(time_to_travel)
print(time_to_travel.unit)

1500.0 s
s


### Example 1 - A simple search for AIA data

First, let's construct a simple search for AIA images using the Fido client. To do this, we have to construct a search query using valid *attributes*. Above, we imported the attributes module and gave it the short name *a*.

Let's search for all AIA files between 06:20 and 06:30 on 2011 June 7, during which there was an M-class solar flare.

In [3]:
example1_search = Fido.search(a.Time('2011-06-07 06:20','2011-06-07 06:30'), a.Instrument('AIA'))

In [4]:
print(example1_search)

Results from 1 Provider:

402 Results from the VSOClient:
   Start Time [1]       End Time [1]    Source ...   Type    Wavelength [2] 
                                               ...              Angstrom    
       str19               str19         str3  ...   str8       float64     
------------------- ------------------- ------ ... -------- ----------------
2011-06-07 06:20:00 2011-06-07 06:20:01    SDO ... FULLDISK   171.0 .. 171.0
2011-06-07 06:20:00 2011-06-07 06:20:01    SDO ... FULLDISK   211.0 .. 211.0
2011-06-07 06:20:02 2011-06-07 06:20:03    SDO ... FULLDISK     94.0 .. 94.0
2011-06-07 06:20:03 2011-06-07 06:20:04    SDO ... FULLDISK   335.0 .. 335.0
2011-06-07 06:20:07 2011-06-07 06:20:08    SDO ... FULLDISK 1700.0 .. 1700.0
2011-06-07 06:20:07 2011-06-07 06:20:08    SDO ... FULLDISK   193.0 .. 193.0
2011-06-07 06:20:08 2011-06-07 06:20:09    SDO ... FULLDISK     94.0 .. 94.0
2011-06-07 06:20:09 2011-06-07 06:20:10    SDO ... FULLDISK   131.0 .. 131.0
2011-06-07 06:20:1

From this summary of the search we can understand a few things. First, the search returned 402 files. We can see the properties of these files, such as the wavelength and time interval. Secondly, we can see that these entries were sourced from the Virtual Solar Observatory (VSO) client.

Let's refine this search to return only files with the 171A filter.

In [5]:
example1_search = Fido.search(a.Time('2011-06-07 06:20','2011-06-07 06:30'),
                              a.Instrument('AIA'), a.Wavelength(171*u.angstrom))

In [6]:
print(example1_search)

Results from 1 Provider:

51 Results from the VSOClient:
   Start Time [1]       End Time [1]    Source ...   Type   Wavelength [2]
                                               ...             Angstrom   
       str19               str19         str3  ...   str8      float64    
------------------- ------------------- ------ ... -------- --------------
2011-06-07 06:20:00 2011-06-07 06:20:01    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:20:12 2011-06-07 06:20:13    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:20:24 2011-06-07 06:20:25    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:20:36 2011-06-07 06:20:37    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:20:48 2011-06-07 06:20:49    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:21:00 2011-06-07 06:21:01    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:21:12 2011-06-07 06:21:13    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:21:24 2011-06-07 06:21:25    SDO ... FULLDISK 171.0 .. 171.0
2011-06-07 06:21:36 2011-06-07 06:21:37    

Now we can see that only 51 results were returned, all for files with the 171A filter. Notice also that we had to specify the wavelength using the astropy unit u.angstroms). Many functions in SunPy use these Units, which remove ambiguity in functions. *HINT*: see what happens if you try to carry out the search using just '171' as the wavelength.

#### Example 1 - downloading the result

The Fido module allows us to easily download the search results using the Fido.fetch function.

Let's download just the first file in our search results:

In [7]:
Fido.fetch(example1_search[:,0], path='.')

[                                        ]========================================


['./aia_lev1_171a_2011_06_07t06_20_00_35z_image_lev1.0.fits']

Now check that the file was correctly downloaded

In [8]:
ls *.fits

aia_lev1_171a_2011_06_07t06_20_00_35z_image_lev1.0.fits
aia_lev1_171a_2011_06_07t06_20_00_35z_image_lev1.fits
go1520110607.fits


### Example 2 - querying multiple instruments simultaneously

We often want to retrieve data from different instruments of missions simultaneously, for example we may want SDO/AIA and STEREO/EUV images together. FIDO allows us to easily construct such searches.

Let's search for AIA and STEREO/EUVI data for the same time interval as before, between 06:20 - 06:30 on 2011 June 7.

In [9]:
example2_search = Fido.search(a.Time('2011-06-07 06:20','2011-06-07 06:30'),
                              a.Instrument('AIA') | a.Instrument('EUVI'))

Using the '|' symbol we can construct queries with multiple options for a given attribute, in this case 'Instrument'.

In [10]:
print(example2_search)

Results from 2 Providers:

402 Results from the VSOClient:
   Start Time [1]       End Time [1]    Source ...   Type    Wavelength [2] 
                                               ...              Angstrom    
       str19               str19         str3  ...   str8       float64     
------------------- ------------------- ------ ... -------- ----------------
2011-06-07 06:20:00 2011-06-07 06:20:01    SDO ... FULLDISK   171.0 .. 171.0
2011-06-07 06:20:00 2011-06-07 06:20:01    SDO ... FULLDISK   211.0 .. 211.0
2011-06-07 06:20:02 2011-06-07 06:20:03    SDO ... FULLDISK     94.0 .. 94.0
2011-06-07 06:20:03 2011-06-07 06:20:04    SDO ... FULLDISK   335.0 .. 335.0
2011-06-07 06:20:07 2011-06-07 06:20:08    SDO ... FULLDISK 1700.0 .. 1700.0
2011-06-07 06:20:07 2011-06-07 06:20:08    SDO ... FULLDISK   193.0 .. 193.0
2011-06-07 06:20:08 2011-06-07 06:20:09    SDO ... FULLDISK     94.0 .. 94.0
2011-06-07 06:20:09 2011-06-07 06:20:10    SDO ... FULLDISK   131.0 .. 131.0
2011-06-07 06:20:

Looking at these results, we can see that we have search result in two blocks: the first block contains all the SDO/AIA search results, and the second block contains the STEREO/EUVI search results. As before, there were 402 AIA files found, and 22 STEREO/EUVI files.

These blocks can be indexed and retrieved separately, for example:

In [11]:
print(example2_search[1])

Results from 1 Provider:

22 Results from the VSOClient:
   Start Time [1]       End Time [1]     Source  ...   Type   Wavelength [2]
                                                 ...             Angstrom   
       str19               str19          str8   ...   str8      float64    
------------------- ------------------- -------- ... -------- --------------
2011-06-07 05:03:55 2011-06-07 09:50:15 STEREO_B ... FULLDISK 171.0 .. 175.0
2011-06-07 06:20:10 2011-06-07 06:20:12 STEREO_B ... FULLDISK 171.0 .. 175.0
2011-06-07 06:20:30 2011-06-07 06:20:38 STEREO_A ... FULLDISK 195.0 .. 195.0
2011-06-07 06:20:55 2011-06-07 06:21:03 STEREO_B ... FULLDISK 195.0 .. 195.0
2011-06-07 06:21:25 2011-06-07 06:21:27 STEREO_B ... FULLDISK 171.0 .. 175.0
2011-06-07 06:21:40 2011-06-07 06:21:44 STEREO_B ... FULLDISK 304.0 .. 304.0
2011-06-07 06:21:55 2011-06-07 06:22:27 STEREO_B ... FULLDISK 284.0 .. 284.0
2011-06-07 06:22:40 2011-06-07 06:22:42 STEREO_B ... FULLDISK 171.0 .. 175.0
2011-06-07 06:23:00

...returns only the STEREO search results block.

As before, we can download these files using the Fido.fetch command. Let's download just the first result from the STEREO results block.

In [12]:
Fido.fetch(example2_search[1,0],path='.')

[                                        ]========================================


['./20110607_050330_n5eub.0.fts']

In [13]:
ls

20110607_050330_n5eub.0.fts
20110607_050330_n5eub.fts
README.md
aia_lev1_171a_2011_06_07t06_20_00_35z_image_lev1.0.fits
aia_lev1_171a_2011_06_07t06_20_00_35z_image_lev1.fits
fido_example.ipynb
go1520110607.fits


### Example 3 - querying multiple clients for different data types simultaneously

In both examples above, Fido returned results exclusively from the VSO client. Crucially though, Fido supports the search for data files from multiple different clients simulaneously. 

As an example, let's construct a query where we want to obtain the AIA data, STEREO/EUVI data, and the GOES/XRS time series data from the 2011 June 7 event simultaneously.

In [14]:
example3_search = Fido.search(a.Time('2011-06-07 06:20','2011-06-07 06:30'),
                              a.Instrument('AIA') | a.Instrument('EUVI') | a.Instrument('GOES'))

In [15]:
print(example3_search)

Results from 3 Providers:

402 Results from the VSOClient:
   Start Time [1]       End Time [1]    Source ...   Type    Wavelength [2] 
                                               ...              Angstrom    
       str19               str19         str3  ...   str8       float64     
------------------- ------------------- ------ ... -------- ----------------
2011-06-07 06:20:00 2011-06-07 06:20:01    SDO ... FULLDISK   171.0 .. 171.0
2011-06-07 06:20:00 2011-06-07 06:20:01    SDO ... FULLDISK   211.0 .. 211.0
2011-06-07 06:20:02 2011-06-07 06:20:03    SDO ... FULLDISK     94.0 .. 94.0
2011-06-07 06:20:03 2011-06-07 06:20:04    SDO ... FULLDISK   335.0 .. 335.0
2011-06-07 06:20:07 2011-06-07 06:20:08    SDO ... FULLDISK 1700.0 .. 1700.0
2011-06-07 06:20:07 2011-06-07 06:20:08    SDO ... FULLDISK   193.0 .. 193.0
2011-06-07 06:20:08 2011-06-07 06:20:09    SDO ... FULLDISK     94.0 .. 94.0
2011-06-07 06:20:09 2011-06-07 06:20:10    SDO ... FULLDISK   131.0 .. 131.0
2011-06-07 06:20:

We can see that this time, we have three blocks of responses. The third block contains results from another client, the GOESClient, which is for GOES/XRS time series data. FIDO automatically communicates with all of the different SunPy clients as required to find the requested data.

In [16]:
example3_search[2]

Start Time,End Time,Source,Instrument,Wavelength
str19,str19,str4,str4,str3
2011-06-07 06:20:00,2011-06-07 06:30:00,nasa,goes,nan


Once again, we can download just the GOES/XRS timeseries data using Fido.fetch.

In [17]:
Fido.fetch(example3_search[2,:],path='.')

[                                        ]========================================


['./go1520110607.0.fits']

In [18]:
ls

20110607_050330_n5eub.0.fts
20110607_050330_n5eub.fts
README.md
aia_lev1_171a_2011_06_07t06_20_00_35z_image_lev1.0.fits
aia_lev1_171a_2011_06_07t06_20_00_35z_image_lev1.fits
fido_example.ipynb
go1520110607.0.fits
go1520110607.fits


### Summary

By the end of this notebook you should have a basic understanding of how to use FIDO, the unified downloader client in SunPy. Here are some key things to remember:
   - The function Fido.search() allows you to search online for data files from a large suite of solar missions.
   - The Fido.fetch() command allows you to download the reults of a search.
   - Searches are constructed using *attributes*. Sometimes these attributes need to be in the form of an *AstroPy Quantity*.
   - Complex queries can be constructed via combinations of attributes - multiple options for an attribute may be used simultaneously.
   - Fido can search for different data types from different sources simultaneously, e.g. timeseries, images and spectra.